## Machine Learning CA1

Student: Charles Franklin Jahn 2020315

Lecturer: David Mcquaid

CCT College Dublin

BSc (Hons) in Computing in IT - 4nd Year

22/04/2023

## References:



## GitHub:
https://github.com/CharlesCCT2020315/CA1_Machine_Learning